<a href="https://colab.research.google.com/github/jacquesbilombe/Plustrategy/blob/main/datafetcher.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For more information on the API, check this [link](https://pypi.org/project/sportradar/).

In [1]:
import os
import sys
import csv
import json
import datetime
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup

In [2]:
def credentials():
  cookies = {
    '_gcl_au': '1.1.1494853104.1715364312',
    '_ga': 'GA1.1.1918003453.1715364313',
    '__qca': 'P0-1691788869-1715364322655',
    'FCNEC': '%5B%5B%22AKsRol9XuTvtcA-KtSeor_fgzdQW58RK_BpfZB7Sdj5I2XedsLxKLTo20tcRe8s8L53_TyoQtuRWeKKXFto4oIVAygDrfxFElND8Xx-JTSyxHbl3W_lqOhjJ7snWZ73Xr-3PupQZsWVMa3yJ19Vv92Tc6i2muqe-rg%3D%3D%22%5D%5D',
    '__gads': 'ID=b5947f16edb0403d:T=1715364320:RT=1715366201:S=ALNI_MaJIyMoeqZaO--UX-FZHBggRDNdJA',
    '__gpi': 'UID=00000a257e3bc349:T=1715364320:RT=1715366201:S=ALNI_MbCel5XYfhe-MDCNwKoU5oysKgFOA',
    '__eoi': 'ID=934a22df78350c96:T=1715364320:RT=1715366201:S=AA-AfjZuN6AaSh_XAQiV6aYPqXtz',
    '_ga_3KF4XTPHC4': 'GS1.1.1715364313.1.1.1715366296.60.0.0',
    '_ga_QH2YGS7BB4': 'GS1.1.1715364313.1.1.1715366296.0.0.0',
    '_ga_HNQ9P9MGZR': 'GS1.1.1715364319.1.1.1715366296.57.0.0'}

  headers = {
      'accept': '*/*',
      'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7,pt-BR;q=0.6,pt;q=0.5,fr-CD;q=0.4,ja-JP;q=0.3,ja;q=0.2',
      'cache-control': 'max-age=0',
      'dnt': '1',
      'if-none-match': 'W/"f9dc1f6f67"',
      'priority': 'u=1, i',
      'referer': 'https://www.sofascore.com/',
      'sec-ch-ua': '"Chromium";v="124", "Google Chrome";v="124", "Not-A.Brand";v="99"',
      'sec-ch-ua-mobile': '?0',
      'sec-ch-ua-platform': '"Linux"',
      'sec-fetch-dest': 'empty',
      'sec-fetch-mode': 'cors',
      'sec-fetch-site': 'same-origin',
      'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36',
      'x-requested-with': '77b62c'}
  return headers,cookies

# Function to extract information from each dictionary and return as a dictionary
def extract_info(d):
    tournament_name = d['tournament']['name']
    category_name = d['tournament']['category']['name']
    status_description = d['status']['description']
    home_team_short_name = d['homeTeam']['shortName']
    away_team_short_name = d['awayTeam']['shortName']
    start_timestamp = d['startTimestamp']
    game_id = str(d['id'])
    slug = str(d['slug'])
    game_details = '/' + str(d['slug']) + '/' + str(d['customId']) + '#id:' + str(d['id'])

    return {
        'Tournament': tournament_name,
        'Country': category_name,
        'Status': status_description,
        'Home Team': home_team_short_name,
        'Away Team': away_team_short_name,
        'Start Timestamp': start_timestamp,
        'Game id': game_id,
        'slug':slug,
        'Game link': 'https://www.sofascore.com' + game_details
    }

def ref_details(referee):
    # Define the URL for referee details
    url = "https://www.sofascore.com/api/v1/event/" + referee

    # Make the request to fetch referee details
    response = requests.get(url,
              cookies=cookies,
              headers=headers)

    # Check if request was successful
    if response.status_code == 200:
        data = response.json()
        data = list(data.values())[0]
        # Extract referee details
        try:
            name = data['referee']['name']
            yellowCards = data['referee']['yellowCards']
            redCards = data['referee']['redCards']
            games = data['referee']['games']
            # Return the referee details
            return {
                'Game id': referee,
                'Referee Name': name,
                'Avg Yellow Cards': np.round(yellowCards/games,2),
                'Avg Red Cards': np.round(redCards/games, 2),
            }
        except:
            return {
                'Game id': referee,
                'Referee Name': None,
                'Avg Yellow Cards': None,
                'Avg Red Cards': None,
            }

    else:
      return {'Game id': referee, 'Name': None, 'Yellow Cards': None, 'Red Cards': None}

def nest_json():# Create nested dictionary
  nested_dict = {}
  for _, row in merged_df.iterrows():
      country = row['Country']
      tournament = row['Tournament']
      game_id = row['Game id']

      if country not in nested_dict:
          nested_dict[country] = {}

      if tournament not in nested_dict[country]:
          nested_dict[country][tournament] = {}

      nested_dict[country][tournament][game_id] = {
          'slug': row['slug'],
          'gametime': row['Start Timestamp'].strftime('%Y-%m-%d %H:%M:%S'),
          'hometeam': row['Home Team'],
          'awayteam': row['Away Team'],
          'Status': row['Status'],
          'referee': {
              'name': row['Referee Name'],
              'Avg YellowCards': row['Avg Yellow Cards'],
              'Avg RedCards': row['Avg Red Cards']
          }
      }

  # Convert nested dictionary to JSON
  json_data = json.dumps(nested_dict, indent=4)

  # Write to a file if needed
  with open('output.json', 'w') as json_file:
      json_file.write(json_data)

In [3]:
headers,cookies = credentials()

today = datetime.datetime.today().date()
today_str = today.strftime('%Y-%m-%d')


response = requests.get(
    'https://www.sofascore.com/api/v1/sport/football/scheduled-events/' + today_str,
    cookies=cookies,
    headers=headers
    )

# Now we can get the data we want
today_games = response.json()


In [4]:
matches = list(today_games.values())
matches = matches[0]

# Apply the extraction function to each dictionary in the list
extracted_data_list = [extract_info(d) for d in matches]

# Create DataFrame from the list of extracted data
df = pd.DataFrame(extracted_data_list)
df['Start Timestamp'] = pd.to_datetime(df['Start Timestamp'], unit='s')

# Extract the date from the 'Start Timestamp' column
df['Date'] = df['Start Timestamp'].dt.date

# Filter the DataFrame by today's date
today = datetime.datetime.today().date()
df = df[df['Date'] == today]

df = df.drop(columns=['Date'])


In [5]:
# Apply the extraction function to each dictionary in the list
ref_list = [ref_details(referee) for referee in df['Game id'].values]

# Create DataFrame from the list of extracted data
df_ref = pd.DataFrame(ref_list)

In [6]:
# Merge both data set
merged_df = pd.merge(df, df_ref, on='Game id')
merged_df.head(5)


,Tournament,Country,Status,Home Team,Away Team,Start Timestamp,Game id,slug,Game link,Referee Name,Avg Yellow Cards,Avg Red Cards
0,Serie A,Italy,Ended,Atalanta,Fiorentina,2024-06-02 16:00:00,12327421,fiorentina-atalanta,https://www.sofascore.com/fiorentina-atalanta/...,Daniele Orsato,5.10,0.08
1,"Ligue 1, Relegation/Promotion Playoffs",France,Ended,Metz,Saint-Étienne,2024-06-02 15:00:00,12290097,saint-etienne-metz,https://www.sofascore.com/saint-etienne-metz/b...,Jérôme Brisard,3.68,0.22
2,"Liga Portugal Betclic, Relegation/Promotion",Portugal,Not started,AVS,Portimonense,2024-06-02 18:45:00,12352936,avs-futebol-sad-portimonense,https://www.sofascore.com/avs-futebol-sad-port...,Joao Pedro Pinheiro,4.75,0.12
3,Brasileirão Série A,Brazil,Ended,Corinthians,Botafogo,2024-06-02 00:00:00,12357219,botafogo-corinthians,https://www.sofascore.com/botafogo-corinthians...,Anderson Daronco,4.32,0.14
4,Brasileirão Série A,Brazil,Not started,Atlético-MG,Bahia,2024-06-02 19:00:00,12357241,atletico-mineiro-bahia,https://www.sofascore.com/atletico-mineiro-bah...,Bruno Arleu de Araujo,5.34,0.18


In [9]:
'''odds = requests.get('https://www.sofascore.com/api/v1/sport/football/odds/1/2024-05-11', cookies=cookies, headers=headers)

# Now we can get the data we want
today_odds = odds.json()
today_odds'''

"odds = requests.get('https://www.sofascore.com/api/v1/sport/football/odds/1/2024-05-11', cookies=cookies, headers=headers)\n\n# Now we can get the data we want\ntoday_odds = odds.json()\ntoday_odds"